# Market Room Type Classifier
In this notebook we experiment the NLP and K-means clustering functions for standarizing the room types from the OTA property scrapped data.

In [11]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

In [12]:
import os
import sys

sys.path.insert(1,"/home/ushan/workspace/rezaware/")
import rezaware as reza
from utils.modules.etl.load import sparkFILEwls as files
from utils.modules.ml.natlang import nlp 
from utils.modules.ml.cluster import points 

''' restart initiate classes '''
if debug:
    import importlib
    reza = importlib.reload(reza)
    rw = importlib.reload(files)
    nlp = importlib.reload(nlp)
    points = importlib.reload(points)
    
__desc__ = "cluster and classify room types"
clsRW = files.FileWorkLoads(desc=__desc__)
clsCL = points.ClusterWorkLoads(desc=__desc__)
clsNLP = nlp.NatLanWorkLoads(desc=__desc__)
print("\nClass initialization and load complete!")

All functional APP-libraries in REZAWARE-package of REZAWARE-module imported successfully!
Some packages in ETL module LOAD package for FILEWORKLOADS function didn't load
No module named 'google'
All functional NATLANWORKLOADS-libraries in NATLANG-package of ML-module imported successfully!
All functional CLUSTERWORKLOADS-libraries in CLUSTER-package of ML-module imported successfully!
FileWorkLoads Class initialization complete
ClusterWorkLoads Class initialization complete
NatLanWorkLoads Class initialization complete

Class initialization and load complete!


## Read data into DataFrame

In [15]:
# __fpath__ = os.path.join("/home/ushan/workspace/rezaware/",
#                         "wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0")
# read_kwargs = {
#     "HEADER":"true",
#     "INFERSCHEMA":"true",
#     "RECURSIVELOOKUP":"true",
#     "TO_PANDAS":True,
# }
# _data = clsRW.read_csv_to_sdf(
#     filesPath = __fpath__,
#     **read_kwargs,
# )
# _data.head(3)

In [13]:
import os
import sys

sys.path.insert(1,"/home/ushan/workspace/rezaware/")
import rezaware as reza
from utils.modules.etl.load import sparkFILEwls as spark
from utils.modules.etl.load.sparkFILEwls import credentials as cred

clsSpark = spark.FileWorkLoads(desc=__desc__)

FileWorkLoads Class initialization complete


In [14]:
clsSpark.storeMode = "local-fs"
clsSpark.storeRoot = "/home/ushan/workspace/rezaware"
# clsSpark.storeRoot = "rezaware-wrangler-source-code"
# clsSpark.storeRoot = "tip-daily-marketcap"   #"rezaware-wrangler-source-code"
print("data_root =",clsSpark.storeRoot)


__as_type__ = "pandas"  # specify data type to return from read file
# folder or key path
__local_folder_path__ = "wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0"
# __aws_folder_path__ = "wrangler/data/hospitality/bookings/scraper/"
# __gcs_folder_path__ = "coingeko"

__local_file_name__ = None  #file name to read from or write data to
# __file_name__ = "scraper-build-scrape-url-list.csv" #"room_descriptions.csv"
# __gcs_file_name__ = "2022-11-21.json"  #file name to read from or write data to
__file_type__ = "csv"   # specify file type to write data to or read from
''' options key value pairs must match the pyspark standard'''
options = {
    "inferSchema":True,
    "header":True,
    "delimiter":",",
}

_data = clsSpark.read_files_to_dtype(
    as_type=__as_type__,
    folder_path=__local_folder_path__,
    file_name=__local_file_name__,
    file_type=__file_type__,
    **options,
)
# if __as_type__ == 'pandas':
#     print("got back {0} rows with return dtype {1}"
#           .format(_data.shape[0],__as_type__))
#     print(_data.head(5))
# elif __as_type__ == 'spark':
#     print("got back {0} rows with return dtype {1}"
#           .format(_data.count(),__as_type__))
#     print(_data.show(5,truncate=False))
# elif __as_type__ == 'dict':
#     print("got back {0} rows with return dtype {1}"
#           .format(len(_data),__as_type__))
#     print(_data)
# else:
#     print('Unrecognised __as_type__' % __as_type__)

data_root = /home/ushan/workspace/rezaware
23/01/18 17:00:02 INFO InMemoryFileIndex: It took 67 ms to list leaf files for 1 paths.
23/01/18 17:00:02 INFO HadoopFSUtils: Listing leaf files and directories in parallel under 204 paths. The first several paths are: file:/home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20023488.2022-09-09.020.csv, file:/home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20088325.2022-09-09.030.csv, file:/home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20015732.2022-09-09.020.csv, file:/home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20050264.2022-09-09.030.csv, file:/home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20015732.2022-09-09.070.csv, file:/home/us

23/01/18 17:00:03 INFO Executor: Finished task 31.0 in stage 9.0 (TID 38). 2020 bytes result sent to driver
23/01/18 17:00:03 INFO TaskSetManager: Starting task 32.0 in stage 9.0 (TID 39) (ushan-Inspiron-5558.lan, executor driver, partition 32, PROCESS_LOCAL, 4595 bytes) taskResourceAssignments Map()
23/01/18 17:00:03 INFO Executor: Running task 32.0 in stage 9.0 (TID 39)
23/01/18 17:00:03 INFO TaskSetManager: Finished task 31.0 in stage 9.0 (TID 38) in 21 ms on ushan-Inspiron-5558.lan (executor driver) (32/204)
23/01/18 17:00:03 INFO Executor: Finished task 32.0 in stage 9.0 (TID 39). 1977 bytes result sent to driver
23/01/18 17:00:03 INFO TaskSetManager: Starting task 33.0 in stage 9.0 (TID 40) (ushan-Inspiron-5558.lan, executor driver, partition 33, PROCESS_LOCAL, 4595 bytes) taskResourceAssignments Map()
23/01/18 17:00:03 INFO TaskSetManager: Finished task 32.0 in stage 9.0 (TID 39) in 23 ms on ushan-Inspiron-5558.lan (executor driver) (33/204)
23/01/18 17:00:03 INFO Executor: Runn

23/01/18 17:00:03 INFO Executor: Finished task 61.0 in stage 9.0 (TID 68). 1977 bytes result sent to driver
23/01/18 17:00:03 INFO TaskSetManager: Starting task 62.0 in stage 9.0 (TID 69) (ushan-Inspiron-5558.lan, executor driver, partition 62, PROCESS_LOCAL, 4595 bytes) taskResourceAssignments Map()
23/01/18 17:00:03 INFO TaskSetManager: Finished task 61.0 in stage 9.0 (TID 68) in 17 ms on ushan-Inspiron-5558.lan (executor driver) (62/204)
23/01/18 17:00:03 INFO Executor: Running task 62.0 in stage 9.0 (TID 69)
23/01/18 17:00:03 INFO Executor: Finished task 62.0 in stage 9.0 (TID 69). 1977 bytes result sent to driver
23/01/18 17:00:03 INFO TaskSetManager: Starting task 63.0 in stage 9.0 (TID 70) (ushan-Inspiron-5558.lan, executor driver, partition 63, PROCESS_LOCAL, 4595 bytes) taskResourceAssignments Map()
23/01/18 17:00:03 INFO TaskSetManager: Finished task 62.0 in stage 9.0 (TID 69) in 19 ms on ushan-Inspiron-5558.lan (executor driver) (63/204)
23/01/18 17:00:03 INFO Executor: Runn

23/01/18 17:00:04 INFO Executor: Finished task 94.0 in stage 9.0 (TID 101). 1977 bytes result sent to driver
23/01/18 17:00:04 INFO TaskSetManager: Starting task 95.0 in stage 9.0 (TID 102) (ushan-Inspiron-5558.lan, executor driver, partition 95, PROCESS_LOCAL, 4595 bytes) taskResourceAssignments Map()
23/01/18 17:00:04 INFO Executor: Running task 95.0 in stage 9.0 (TID 102)
23/01/18 17:00:04 INFO TaskSetManager: Finished task 94.0 in stage 9.0 (TID 101) in 11 ms on ushan-Inspiron-5558.lan (executor driver) (95/204)
23/01/18 17:00:04 INFO Executor: Finished task 95.0 in stage 9.0 (TID 102). 1977 bytes result sent to driver
23/01/18 17:00:04 INFO TaskSetManager: Starting task 96.0 in stage 9.0 (TID 103) (ushan-Inspiron-5558.lan, executor driver, partition 96, PROCESS_LOCAL, 4595 bytes) taskResourceAssignments Map()
23/01/18 17:00:04 INFO TaskSetManager: Finished task 95.0 in stage 9.0 (TID 102) in 13 ms on ushan-Inspiron-5558.lan (executor driver) (96/204)
23/01/18 17:00:04 INFO Executo

23/01/18 17:00:04 INFO Executor: Finished task 110.0 in stage 9.0 (TID 117). 1977 bytes result sent to driver
23/01/18 17:00:04 INFO TaskSetManager: Starting task 111.0 in stage 9.0 (TID 118) (ushan-Inspiron-5558.lan, executor driver, partition 111, PROCESS_LOCAL, 4595 bytes) taskResourceAssignments Map()
23/01/18 17:00:04 INFO TaskSetManager: Finished task 110.0 in stage 9.0 (TID 117) in 14 ms on ushan-Inspiron-5558.lan (executor driver) (111/204)
23/01/18 17:00:04 INFO Executor: Running task 111.0 in stage 9.0 (TID 118)
23/01/18 17:00:04 INFO Executor: Finished task 111.0 in stage 9.0 (TID 118). 1977 bytes result sent to driver
23/01/18 17:00:04 INFO TaskSetManager: Starting task 112.0 in stage 9.0 (TID 119) (ushan-Inspiron-5558.lan, executor driver, partition 112, PROCESS_LOCAL, 4595 bytes) taskResourceAssignments Map()
23/01/18 17:00:04 INFO TaskSetManager: Finished task 111.0 in stage 9.0 (TID 118) in 22 ms on ushan-Inspiron-5558.lan (executor driver) (112/204)
23/01/18 17:00:04 I

...

23/01/18 17:00:09 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20030916.2022-09-09.000.csv, range: 0-6548, partition values: [empty row]
23/01/18 17:00:09 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20030916.2022-09-09.020.csv, range: 0-6548, partition values: [empty row]
23/01/18 17:00:09 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20030916.2022-09-09.010.csv, range: 0-6548, partition values: [empty row]
23/01/18 17:00:09 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20061717.2022-09-09.090.csv, range: 0-5904, partition values: [empty row]
23/01/18 17:

23/01/18 17:00:09 INFO TaskSetManager: Starting task 0.0 in stage 18.0 (TID 235) (ushan-Inspiron-5558.lan, executor driver, partition 0, PROCESS_LOCAL, 10937 bytes) taskResourceAssignments Map()
23/01/18 17:00:09 INFO Executor: Running task 0.0 in stage 18.0 (TID 235)
23/01/18 17:00:09 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20023488.2022-09-09.230.csv, range: 0-9079, partition values: [empty row]
23/01/18 17:00:10 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20131185.2022-09-09.070.csv, range: 0-8658, partition values: [empty row]
23/01/18 17:00:10 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20131185.2022-09-09.050.csv, range: 0-8650, partition values: [e

23/01/18 17:00:10 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20015732.2022-09-09.040.csv, range: 0-7321, partition values: [empty row]
23/01/18 17:00:10 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20050264.2022-09-09.010.csv, range: 0-7299, partition values: [empty row]
23/01/18 17:00:10 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20079110.2022-09-09.090.csv, range: 0-7274, partition values: [empty row]
23/01/18 17:00:10 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20050264.2022-09-09.000.csv, range: 0-7255, partition values: [empty row]
23/01/18 17:

23/01/18 17:00:11 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20021296.2022-09-09.090.csv, range: 0-6810, partition values: [empty row]
23/01/18 17:00:11 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20015732.2022-09-09.100.csv, range: 0-6804, partition values: [empty row]
23/01/18 17:00:11 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20033173.2022-09-09.080.csv, range: 0-6796, partition values: [empty row]
23/01/18 17:00:11 INFO FileScanRDD: Reading File path: file:///home/ushan/workspace/rezaware/wrangler/data/ota/scraper/hospitality/bookings/rates/2022-10-4-13-0/booking.com.20079110.2022-09-09.120.csv, range: 0-6795, partition values: [empty row]
23/01/18 17:

23/01/18 17:00:11 INFO BlockManagerInfo: Removed broadcast_24_piece0 on ushan-Inspiron-5558.lan:45609 in memory (size: 6.1 KiB, free: 434.3 MiB)


In [18]:
_data.tail(2)

,Search DateTime,Checkin Date,Property Name,Room Type,Room Rate,Review Score,Destination ID,Location Desc,Other Info
5085,2022-09-09 16:14:05.613613,2022-09-09,"@ Marbella Lane Mountain View Studio, steps fr...",Studio Apartment,US$296,7.4,20030916,"Waikiki, HonoluluShow on map3.4 miles from centre",Studio ApartmentEntire apartment • 1 bedroom •...
5086,2022-09-09 16:14:05.613613,2022-09-09,THE TIKI SUITE at WAIKIKI BANYAN,One-Bedroom Apartment,US$225,9.0,20030916,"Waikiki, HonoluluShow on map3.3 miles from centre",One-Bedroom ApartmentEntire apartment • 1 bedr...


In [16]:
clsSpark.storeRoot

'/home/ushan/workspace/rezaware'

In [24]:
_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5087 entries, 0 to 5086
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Search DateTime  5087 non-null   datetime64[ns]
 1   Checkin Date     5087 non-null   datetime64[ns]
 2   Property Name    5087 non-null   object        
 3   Room Type        5087 non-null   object        
 4   Room Rate        5087 non-null   object        
 5   Review Score     5087 non-null   float64       
 6   Destination ID   5087 non-null   int32         
 7   Location Desc    5087 non-null   object        
 8   Other Info       5087 non-null   object        
dtypes: datetime64[ns](2), float64(1), int32(1), object(5)
memory usage: 337.9+ KB
23/01/18 17:17:19 INFO BlockManagerInfo: Removed broadcast_10_piece0 on ushan-Inspiron-5558.lan:45609 in memory (size: 34.2 KiB, free: 434.4 MiB)
23/01/18 17:17:19 INFO BlockManagerInfo: Removed broadcast_23_piece0 on ushan-Insp

In [26]:
_data.rename(columns = {'Search DateTime':'search_dt' , 'Checkin Date':'checkin_date', 'Property Name':'property_name' ,'Room Type':'room_type' ,'Room Rate':'room_rate' ,'Review Score':'review_score' ,'Destination ID':'destination_ID' ,'Location Desc':'location_desc' ,'Other Info': 'other_info'    }, inplace = True)



## Get Week No of records

In [21]:
import pandas as pd

In [30]:
_data['search_dt'] = pd.to_datetime(_data['search_dt']).dt.date
_data['checkin_date'] = pd.to_datetime(_data['checkin_date']).dt.date

_data= _data.dropna()

_data['checkin_date'] = pd.to_datetime(_data['checkin_date'])
_data['search_dt'] = pd.to_datetime(_data['search_dt'])

_data['Date_gap'] = _data['checkin_date'] - _data['search_dt']
 # remove 'days' in 'Date_gap' column
_data['Date_gap'] = _data['Date_gap'].astype(str)
_data["Date_gap"]= _data["Date_gap"].replace( r"days","", regex=True)

In [ ]:
_data=_data[_data.room_rate != 0]
_data = _data.reset_index()

In [37]:
_data.head(2)

,search_dt,checkin_date,property_name,room_type,room_rate,review_score,destination_ID,location_desc,other_info,Date_gap
0,2022-09-09,2022-09-09,Marriott's Grande Vista,King Room with Sofa Bed - Non-Smoking,US$169,8.7,20023488,"Sea World Orlando Area, OrlandoShow on map10.6...",King Room with Sofa Bed - Non-Smoking2 beds (1...,0
1,2022-09-09,2022-09-09,Embassy Suites Orlando - North,King Suite-Hearing Access,US$169,7.9,20023488,OrlandoShow on map8.9 miles from centre,King Suite-Hearing AccessPrivate suite2 beds (...,0


## Get room type embeddings 

In [42]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('distilbert-base-nli-mean-tokens')


corpus = list(_data['room_type'])

corpus_embeddings = embedder.encode(corpus)


In [44]:
# _se_props = {
#     "NOSTOPWORDS":True,
#     "LOWER":True,
# }
# _sent,_emb = clsNLP.get_sentence_embeddings(
#     sentences=_data['property_name'],   # list of word ngrams
#     model_name='distilbert-base-nli-mean-tokens',   # https://www.sbert.net/docs/pretrained_models.html
#     kwargs=_se_props

# )
# print(_sent, "\n\n", _emb)

## Label data with kmeans clusters

In [161]:
# __category__ = "KMEANS"
# __columns__ = ['destination_id','review_score']
# clust_props = {
#     "NCLUSTERS":10,
#     "MAXITERATIONS":200,
#     "CENTROIDINIT":5,
#     "RANDOMSTATE":0,
# }
# _cl_data = clsCL.cluster_n_label_data(
#     data = _emb,#_data.dropna(),
#     category = __category__,
#     columns = __columns__,
#     **clust_props
# )
# clsCL.clusters

<class 'numpy.ndarray'>


array([2, 9, 9, ..., 8, 9, 9], dtype=int32)

In [47]:
from sklearn.cluster import KMeans

In [48]:

num_clusters = 10
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_
cluster_assignment

array([1, 4, 7, ..., 7, 7, 0], dtype=int32)

In [51]:
cluster_df = pd.DataFrame(corpus, columns = ['corpus'])
cluster_df['cluster'] = cluster_assignment
cluster_df.head()

,corpus,cluster
0,King Room with Sofa Bed - Non-Smoking,1
1,King Suite-Hearing Access,4
2,Apartment,7
3,"Guest room, 1 King Bed",0
4,King Room,4


## Combine dataframes with updated room types

In [55]:
# Merge two Dataframes on index of both the dataframes
_data= _data.merge(cluster_df, left_index=True, right_index=True)

In [57]:
_data=_data.drop([ 'index', ], axis=1)

In [58]:
_data['new_room_type'] = ''
room_list=['King suite studio Smoking- city view',
'King suite standard studio- 1 Double Delux bed - No smoking',
'Queen suite - 2 standard beds-Guest Sofa',
'Queen superior suite studio with 2 beds',
'Queen suit Delux -2 Double beds - Non Smoking',
'King standard superior',
'Delux King suite Double bed-city View',
'Double Standard delux room 2 Beds - city view',
'King - Access Disability ',
'King suite with Sofa Dormitory - 1 Bed']


for i in range(10):    
    _data['new_room_type'][_data["cluster"]==i]=room_list[i]

In [60]:
_data.head()

,search_dt,checkin_date,property_name,room_type,room_rate,review_score,destination_ID,location_desc,other_info,Date_gap,corpus,cluster,new_room_type
0,2022-09-09,2022-09-09,Marriott's Grande Vista,King Room with Sofa Bed - Non-Smoking,US$169,8.7,20023488,"Sea World Orlando Area, OrlandoShow on map10.6...",King Room with Sofa Bed - Non-Smoking2 beds (1...,0,King Room with Sofa Bed - Non-Smoking,1,King suite standard studio- 1 Double Delux bed...
1,2022-09-09,2022-09-09,Embassy Suites Orlando - North,King Suite-Hearing Access,US$169,7.9,20023488,OrlandoShow on map8.9 miles from centre,King Suite-Hearing AccessPrivate suite2 beds (...,0,King Suite-Hearing Access,4,Queen suit Delux -2 Double beds - Non Smoking
2,2022-09-09,2022-09-09,Cozy 1BR with Two Queen Beds - Pool and Hot Tu...,Apartment,US$77,5.0,20023488,"International Drive, OrlandoShow on map7.7 mil...",ApartmentEntire apartment • 1 bedroom • 1 livi...,0,Apartment,7,Double Standard delux room 2 Beds - city view
3,2022-09-09,2022-09-09,"Grand Bohemian Hotel Orlando, Autograph Collec...","Guest room, 1 King Bed",US$229,8.5,20023488,"Downtown Orlando, OrlandoShow on map0.4 miles ...","Guest room, 1 King Bed1 extra-large double bed...",0,"Guest room, 1 King Bed",0,King suite studio Smoking- city view
4,2022-09-09,2022-09-09,Quality Inn & Suites Near the Theme Parks,King Room,US$149,7.7,20023488,"International Drive, OrlandoShow on map5.6 mil...",King Room1 extra-large double bedBreakfast inc...,0,King Room,4,Queen suit Delux -2 Double beds - Non Smoking
